# Работа самозапрашивающего ретривера


<!--
:::info

Head to [Integrations](/docs/integrations/retrievers/self_query) for documentation on vector stores with built-in support for self-querying.

:::
-->

Самозапрашивающий ретривер - это ретривер, способный обращаться к самому себе.
Такой ретривер принимает запрос на естественном языке, преобразует его в структурированный запрос с помощью LLM-цепочки, после чего применят полученный запрос к заданному векторному хранилищу
Это позволяет ретриверу как использовать запрос пользователя для семантического поиска по содержимому документов, так и применять извлеченные из запроса фильтры по метаданным хранимых документов.

![](../../static/img/self_querying.jpg)

```mermaid
flowchart LR
    A([«Что Петя сказал
       про Колю?»])
    B("Конструктор
        запроса")
    C([Запрос содержит «Коля»
      фильтр author == «Петя»])
    D("Преобразователь
        запроса")
    E([Найди «Коля»,
      где автор «Петя»])
    F[("Векторное
        хранилище")]
    style A fill:stroke:stroke-width:2px,color:#fff,stroke-dasharray: 5 5
    style C fill:stroke:stroke-width:2px,color:#fff,stroke-dasharray: 5 5
    style E fill:stroke:stroke-width:2px,color:#fff,stroke-dasharray: 5 5
    A --> B
    B --> C
    C --> D
    D --> E
    E --> F
```

## Начало работы

Для демонстрации в разделе используется [векторное хранилище Chromа](https://docs.trychroma.com/) и набор документов, которые содержат краткое описание фильмов.

Для работы самозапрашивающего ретривера нужно установить пакет `lark`.

Установите необходимые зависимости.

In [1]:
pip install --upgrade --quiet lark gigachain-community gigachain-chroma

In [2]:
from langchain_chroma import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_core.documents import Document

docs = [
    Document(
        page_content="Трагедия войны глазами солдатской невесты",
        metadata={"year": 1957, "rating": 8.7, "genre": "драма"},
    ),
    Document(
        page_content="Заурядный семьянин Василий Кузякин заводит роман с эффектной коллегой",
        metadata={"year": 1985, "director": "Владимир Меньшов", "rating": 8.2},
    ),
    Document(
        page_content="Встреча Алисы и Коли становится началом ярких приключений, в которых они вступят в схватку с космическими пиратами",
        metadata={"year": 2024, "director": "Александр Андрющенко", "rating": 7.2},
    ),
    Document(
        page_content="Легендарный советский шпионский сериал Татьяны Лиозновой о штандартенфюрере Штирлице",
        metadata={"year": 1973, "director": "Татьяна Лиознова", "rating": 8.3},
    ),
    Document(
        page_content="Непутевый богатырь случайно упускает орду тугар со всем золотом Ростова и теперь спешит догнать и одолеть варваров",
        metadata={"year": 2004, "genre": "мультфильм"},
    ),
    Document(
        page_content="Мистическое путешествие через Зону к комнате, где исполняются желания",
        metadata={
            "year": 1979,
            "director": "Андрей Тарковский",
            "genre": "фантастика",
            "rating": 9.9,
        },
    ),
]

embedding = GigaChatEmbeddings(
    credentials="авторизационные_данные",
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False,
)

vectorstore = Chroma.from_documents(docs, embedding)

:::note

Авторизационные данные — строка, полученная в результате кодирования в Base64 клиентского идентификатора (Client ID) и ключа (Client Secret) API. Вы можете использовать готовые данные из личного кабинета или самостоятельно закодировать идентификатор и ключ.

Пример строки авторизационных данных:

```text
MjIzODA0YTktMDU3OC00MTZmLWI4MWYtYzUwNjg3Njk4MzMzOjljMTI2MGQyLTFkNTEtNGRkOS05ZGVhLTBhNjAzZTdjZjQ3Mw==
```

Идентификатор, ключ и авторизационные данные вы можете получить после создания проекта GigaChat API:

* [для физических лиц](https://developers.sber.ru/docs/ru/gigachat/individuals-quickstart#shag-1-sozdayte-proekt-giga-chat-api);
* [для ИП и юридических лиц](https://developers.sber.ru/docs/ru/gigachat/legal-quickstart#shag-1-otpravte-zayavku-na-dostup-k-proektu-giga-chat-api).

:::

## Создание самозапршивающего ретривера

Добавьте описание фильтров, которые поддерживают документы, и инициализируйте ретривер.

In [3]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_community.chat_models.gigachat import GigaChat
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="Жанр кино или мультфильма. Возможные значения ['фантастика', 'комедия', 'драма', 'триллер', 'мелодрама', 'экшн', 'мультфильм']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="Год выпуска",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="Имя режиссера",
        type="string",
    ),
    AttributeInfo(
        name="rating",
        description="Рейтинг кино или мультфильма от 1 до 10",
        type="float",
    ),
]
document_content_description = "Краткое описание кино или мультфильма"
llm = GigaChat(
    credentials="авторизационные_данные",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

### Использование ретривера

Теперь вы можете проверить работу созданного ретривера.

In [15]:
# Пример фильтра
retriever.invoke("Хочу посмотреть фильм с рейтингом больше 8.5")

[Document(page_content='Трагедия войны глазами солдатской невесты', metadata={'genre': 'драма', 'rating': 8.7, 'year': 1957}),
 Document(page_content='Мистическое путешествие через Зону к комнате, где исполняются желания', metadata={'director': 'Андрей Тарковский', 'genre': 'фантастика', 'rating': 9.9, 'year': 1979})]

In [8]:
# Пример запроса и фильтра
retriever.invoke("Фильм Татьяны Лиозновой про Штирлица")

[Document(page_content='Легендарный советский шпионский сериал Татьяны Лиозновой о штандартенфюрере Штирлице', metadata={'director': 'Татьяна Лиознова', 'rating': 8.3, 'year': 1973})]

In [9]:
# Пример составного фильтра
retriever.invoke(
    "Есть какие-нибудь высокооцененные (с рейтингом выше 8.5) фантастические фильмы?"
)

[Document(page_content='Мистическое путешествие через Зону к комнате, где исполняются желания', metadata={'director': 'Андрей Тарковский', 'genre': 'фантастика', 'rating': 9.9, 'year': 1979})]

In [28]:
# Пример запроса и составного фильтра
retriever.invoke("мультфильм про богатыря, который вышел с 1999 по 2007")

[Document(page_content='Непутевый богатырь случайно упускает орду тугар со всем золотом Ростова и теперь спешит догнать и одолеть варваров', metadata={'genre': 'мультфильм', 'year': 2004})]

### Ограничение количества запрашиваемых документов

Используйте параметр `enable_limit=True` чтобы задать количество документов, которые нужно получить.

In [11]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
)

# Пример релевантного запроса
retriever.invoke("Один фильм про войну")

[Document(page_content='Трагедия войны глазами солдатской невесты', metadata={'genre': 'драма', 'rating': 8.7, 'year': 1957})]

## Создание ретривера с помощью LCEL

Вы можете переписать свой ретривер с использованием LCEL.
Реализация на LCEL даст больше контроля за работой ретривера и информации о том, что происходит «под капотом».

Сначала создайте цепочку, которая будет отвечать за формирование запроса.
Эта цепочка будет преобразовывать запрос пользователя в структурированный запрос (объект `StructuredQuery`), который содержит заданные пользователем фильтры.

Для создания промпта и парсера в примере используются вспомогательные функции `get_query_constructor_prompt()` и `from_components()` соответственно.

In [12]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

Теперь вы можете посмотреть какой промпт используется при вызове модели:

In [13]:
print(prompt.format(query="заглушка"))

Твоя задача - структурировать запрос пользователя, чтобы он соответствовал схеме запроса, представленной ниже.

<< Схема структурированного запроса >>
При ответе используйте фрагмент кода markdown с объектом JSON, отформатированным по следующей схеме:

```json
{
    "query": string \ текстовая строка для сравнения с содержимым документа
    "filter": string \ логическое условие для фильтрации документов
}
```

Строка запроса должна содержать только текст, который ожидается в содержимом документов. Любые условия в фильтре не должны упоминаться в запросе.

Логическое условие состоит из одного или нескольких операторов сравнения и логических операций.

Оператор сравнения имеет форму: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): оператор сравнения
- `attr` (string):  имя атрибута, к которому применяется сравнение
- `val` (string): значение для сравнения

Логическая операция имеет форму `op(statement1, statement2, ...)`:
- `op` (and | or | not): л

Результат работы цепочки:

In [14]:
query_constructor.invoke(
    {"query": "Научнофантастические фильмы Андрея Тарковского снятые в семидесятых"}
)

StructuredQuery(query='научно-фантастический фильм', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='фантастика'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Андрей Тарковский'), Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year', value=1970), Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='year', value=1979)]), limit=None)

Конструктор запросов — ключевой элеменет самозапрашивающего ретривера.
Чтобы добиться хорошей работы конструктора, зачастую требуется настройка промпта, использование образцов в промпте и описание атрибутов.

Другим важным элементом является преобразователь структурированного запроса (*транслятор*).
Он преобразует объект `StructuredQuery` в фильтр метаданных согласно синтаксису векторного хранилища, которое вы используете.
GigaChain предоставляет доступ к преобразователям, встроенным в LangChain.
Подробнее о них можно прочитать в [официальной документации](/docs/integrations/retrievers/self_query).

In [15]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=ChromaTranslator(),
)

In [16]:
retriever.invoke(
    "Есть какие-нибудь высокооцененные (с рейтингом выше 8.5) фантастические фильмы?"
)

[Document(page_content='Мистическое путешествие через Зону к комнате, где исполняются желания', metadata={'director': 'Андрей Тарковский', 'genre': 'фантастика', 'rating': 9.9, 'year': 1979})]